In [ ]:
pip install SimpleITK
pip install dltk
pip install nipype

In [ ]:
from nipype.interfaces import fsl
from nipype.testing import example_data
import nibabel as nib
import numpy as np

import SimpleITK as sitk
import os
import pandas as pd

from matplotlib import pyplot as plt

from dltk.io.augmentation import *
from dltk.io.preprocessing import *

%matplotlib inline

import skimage
from skimage import data
from skimage.transform import resize

# Generate CSV File

In [ ]:
# If there is no a csv file for the data, it can be created by using the code below. Data should be in folders according to their labels.

In [ ]:
def get_images(dir_name=r'/content/drive/Othercomputers/MSI/Documents/BitirmeTezi/Uygulama/ADNI1_Baseline_3T/ADNI-yedek/test'):
    
    Images = []
    Classes = []
    m_names=[]
    #for j in ['/train','/test']:
    for label_name in os.listdir(dir_name):
        for img_file_name in os.listdir('/'.join([dir_name, label_name])):

            m_names.append([img_file_name, label_name])

    return m_names

In [ ]:
m_names= get_images("/content/drive/Othercomputers/MSI/Documents/BitirmeTezi/Uygulama/ADNI1_Baseline_3T/ADNI-yedek/test")

df = pd.DataFrame(m_names, columns=['img_file_name', 'label_name']) # create a dataframe from match list
df.to_csv("test.csv", index=False) # create csv from df

In [ ]:
m_names= get_images("/content/drive/Othercomputers/MSI/Documents/BitirmeTezi/Uygulama/ADNI1_Baseline_3T/ADNI-yedek/train")

df = pd.DataFrame(m_names, columns=['img_file_name', 'label_name']) # create a dataframe from match list
df.to_csv("train.csv", index=False) # create csv from df

# Prepare The Data

In [ ]:
import gzip
import shutil
#Unzip the FSL zip files
for filename in os.listdir('/content/drive/Othercomputers/MSI/Documents/BitirmeTezi/Uygulama/ADNI1_Baseline_3T/ADNI-yedek/skull_stripped_test'): 
  with gzip.open(os.path.join('/content/drive/Othercomputers/MSI/Documents/BitirmeTezi/Uygulama/ADNI1_Baseline_3T/ADNI-yedek/skull_stripped_test', filename), 'rb') as f_in:
      with open(os.path.join('/content/drive/MyDrive/preprocessed/test_unzipped', filename.replace(".gz","")), 'wb') as f_out:
         shutil.copyfileobj(f_in, f_out)

In [ ]:
for filename in os.listdir('/content/drive/Othercomputers/MSI/Documents/BitirmeTezi/Uygulama/ADNI1_Baseline_3T/ADNI-yedek/skull_stripped_train'): 
  with gzip.open(os.path.join('/content/drive/Othercomputers/MSI/Documents/BitirmeTezi/Uygulama/ADNI1_Baseline_3T/ADNI-yedek/skull_stripped_train', filename), 'rb') as f_in:
      with open(os.path.join('/content/drive/MyDrive/preprocessed/train_unzipped', filename.replace(".gz","")), 'wb') as f_out:
         shutil.copyfileobj(f_in, f_out)

In [ ]:
#For The Training Data

# prepare and organize the images
path='/content/drive/MyDrive/preprocessed/train_unzipped'
for filename in os.listdir(path):   
        complete_file_path = os.path.join(path, filename)  # filename
        # load sitk image
        sitk_moving = sitk.ReadImage(complete_file_path)
        #sitk_moving = resample_img(sitk_moving)
        t1 = sitk.GetArrayFromImage(sitk_moving)
        # Resizing image to [128, 256, 256] required padding
        #t1_padded = resize_image_with_crop_or_pad(t1, [128, 128, 128], mode='symmetric')
        sitk_moving=skimage.transform.resize(t1, [128, 128, 128])
        t1_norm_zo = normalise_zero_one(sitk_moving) #Diğer koddan alındı
        
        # Crop to [64, 64, 64]
        #t1_cropped = resize_image_with_crop_or_pad(t1, [64, 64, 64], mode='symmetric')


        # Add a feature dimension and normalise
        #t1_norm = np.expand_dims(normalise_one_one(t1_slice), axis=-1)

        # Randomly flip the image along axis 1
        #t1_flipped = flip(t1_norm_zo.copy(), axis=1)

        # Add a Gaussian offset (independently for each channel)
        #t1_offset = add_gaussian_offset(t1_norm_zo.copy(), sigma=0.5)

        # Add Gaussian noise
        #t1_noise = add_gaussian_noise(t1_norm_zo.copy(), sigma=0.25)

        # Elastic transforms 
        #t1_trans_low_s = elastic_transform(t1_norm.copy(), alpha=[1, 1e5, 1e5], sigma=[1, 10, 10])
        #t1_trans_high_s = elastic_transform(t1_norm.copy(), alpha=[10, 2e6, 2e6], sigma=[1, 25, 25])


        new_image=sitk.GetImageFromArray(t1_norm_zo)
        
        # prepare the destination path
        complete_new_path = os.path.join('/content/drive/MyDrive/preprocessed/train', 
                                        filename)  #filename
        sitk.WriteImage(new_image, complete_new_path)

In [ ]:
#For The Test Data


# prepare and organize the images
path='/content/drive/MyDrive/preprocessed/test_unzipped'
#for files in os.walk(DATABASE):
for filename in os.listdir(path):   
    #for filename in os.listdir(os.path.join(path, classname)):
        complete_file_path = os.path.join(path, filename)  # filename
        # load sitk image
        sitk_moving = sitk.ReadImage(complete_file_path)
        #sitk_moving = resample_img(sitk_moving)
        # Resizing image to [128, 256, 256] required padding
        
        t1 = sitk.GetArrayFromImage(sitk_moving)
        #t1_padded = resize_image_with_crop_or_pad(t1, [128, 128, 128], mode='symmetric')

        sitk_moving=skimage.transform.resize(t1, [128, 128, 128])
        t1_norm_zo = normalise_zero_one(sitk_moving) #Diğer koddan alındı
        # Crop to [64, 64, 64]
        #t1_cropped = resize_image_with_crop_or_pad(t1, [64, 64, 64], mode='symmetric')


        # Add a feature dimension and normalise
        #t1_norm = np.expand_dims(normalise_one_one(t1_slice), axis=-1)

        # Randomly flip the image along axis 1
        #t1_flipped = flip(t1_norm_zo.copy(), axis=1)

        # Add a Gaussian offset (independently for each channel)
        #t1_offset = add_gaussian_offset(t1_norm_zo.copy(), sigma=0.5)

        # Add Gaussian noise
        #t1_noise = add_gaussian_noise(t1_norm_zo.copy(), sigma=0.25)

        # Elastic transforms 
        #t1_trans_low_s = elastic_transform(t1_norm.copy(), alpha=[1, 1e5, 1e5], sigma=[1, 10, 10])
        #t1_trans_high_s = elastic_transform(t1_norm.copy(), alpha=[10, 2e6, 2e6], sigma=[1, 25, 25])


        new_image=sitk.GetImageFromArray(t1_norm_zo)


        # prepare the destination path
        complete_new_path = os.path.join('/content/drive/MyDrive/preprocessed/test', 
                                        filename)  #filename
        sitk.WriteImage(new_image, complete_new_path)


In [ ]:
rootdir = r'/content/drive/MyDrive/preprocessed/train'
str = "_brain"
for filename in os.listdir(rootdir):
    if str in filename:    
        filepath = os.path.join(rootdir, filename)
        newfilepath = os.path.join(rootdir, filename.replace(str, ""))
        os.rename(filepath, newfilepath)

In [ ]:
rootdir = r'/content/drive/MyDrive/preprocessed/test'
str = "_brain"
for filename in os.listdir(rootdir):
    if str in filename:    
        filepath = os.path.join(rootdir, filename)
        newfilepath = os.path.join(rootdir, filename.replace(str, ""))
        os.rename(filepath, newfilepath)

# Training Part

In [ ]:
pip install monai
pip install torchinfo

In [ ]:
import logging
import os
import sys

import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import pandas as pd

import monai
from monai.data import ImageDataset
from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, EnsureType


In [ ]:
from tensorflow import summary
%load_ext tensorboard

In [ ]:
label_to_class = {
    'NC': 0,
    'MCI': 1,
    'AD': 2,
    #Extra classes can be added
    #'NewClass':3
    
}
class_to_label = {v: k for k, v in label_to_class.items()}
n_classes = len(label_to_class)


In [ ]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

images = []
labels=[]
image_labels= pd.read_csv("/content/drive/MyDrive/train.csv")
image_labels = image_labels.sample(frac=1).reset_index(drop=True)

for index in range(159):
  labels.append(label_to_class[image_labels.iloc[index, 1]])
  images.append(os.path.join("/content/drive/MyDrive/preprocessed/train",image_labels.iloc[index,0])) 

labels = np.array(labels)
image_file_names=image_labels.iloc[index,0]

In [ ]:
print(labels)
labels.shape

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
num_epochs=50
#split için değişken oluştur

# Define Training and Validation transforms
train_transforms = Compose([ScaleIntensity(), AddChannel(), RandRotate90(), EnsureType()])  
val_transforms = Compose([ScaleIntensity(), AddChannel(),  EnsureType()])   

# Define image dataset, data loader
check_ds = ImageDataset(image_files=images, labels=labels,transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=2, num_workers=2, pin_memory=torch.cuda.is_available())
im, label = monai.utils.misc.first(check_loader)
print(type(im), im.shape, label)

# create a training data loader
train_ds = ImageDataset(image_files=images[:144], labels=labels[:144], transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=7, shuffle=True, num_workers=2, pin_memory=torch.cuda.is_available())

# create a validation data loader
val_ds = ImageDataset(image_files=images[-15:], labels=labels[-15:], transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=5, shuffle=True, num_workers=2, pin_memory=torch.cuda.is_available())

# Create DenseNet121, CrossEntropyLoss and Adam optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = monai.networks.nets.DenseNet121(spatial_dims=3,in_channels=1, out_channels=3 ).to(device)  
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-7 ) 
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-5 , steps_per_epoch=len(train_loader), epochs=num_epochs)
# start a typical PyTorch training
val_interval = 2
best_metric = -1
epoch_loss_values = list()
metric_values = list()
writer = SummaryWriter()

In [ ]:
from torchinfo import summary

batch_size = 16
summary(model)

In [ ]:
%tensorboard --logdir /content/runs/RunFileMustBeAdded

In [ ]:
validation_model_training_loss_ls = []
validation_model_training_accuracy_ls = []

for epoch in range(num_epochs):
      print("-" * 10)
      print(f"epoch {epoch + 1}/{num_epochs}")
      model.train()
      epoch_loss = 0
      epoch_val_loss=0
      step = 0
      val_step=0
      for batch_data in train_loader:
          step += 1
          inputs, labels = batch_data[0].to(device), batch_data[1].to(device)
          optimizer.zero_grad()
          outputs = model(inputs)
          loss = loss_function(outputs, labels)
          loss.backward()
          optimizer.step()
          scheduler.step()
          curr_lr = optimizer.param_groups[0]["lr"]
          epoch_loss += loss.item()
          epoch_len = len(train_ds) // train_loader.batch_size
          print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
          writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)
          writer.add_scalar("learning_rate",  curr_lr, epoch_len * epoch + step)
      epoch_loss /= step
      epoch_loss_values.append(epoch_loss)
      print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
      #writer.add_scalar("total_train_loss", epoch_loss, epoch_len * epoch + step)

      if (epoch + 1) % val_interval == 0:
          model.eval()
          with torch.no_grad():
              num_correct = 0.0
              metric_count = 0
              for val_data in val_loader:
                  val_step=val_step+1
                  val_images, val_labels = val_data[0].to(device), val_data[1].to(device)
                  val_outputs = model(val_images)
                  value = torch.eq(val_outputs.argmax(dim=1), val_labels)
                  metric_count += len(value)
                  num_correct += value.sum().item()
                  
                  validation_loss_local_train_fn = loss_function(val_outputs, val_labels)##Doğru mu?
                  #validation_model_training_loss_ls.append(validation_loss_local_train_fn.item())##
                  epoch_val_loss += validation_loss_local_train_fn.item()
                  print(f"{step}/{epoch_len}, val_loss: {validation_loss_local_train_fn.item():.4f}")
                  writer.add_scalar("val_loss", validation_loss_local_train_fn.item(), epoch_len * epoch + step)
              metric = num_correct / metric_count
              metric_values.append(metric)
              epoch_val_loss /= val_step
              epoch_loss_values.append(epoch_val_loss)
              print(f"epoch {epoch + 1} average validation loss: {epoch_val_loss:.4f}")
              
              if metric > best_metric:
                  best_metric = metric
                  best_metric_epoch = epoch + 1
                  torch.save(model.state_dict(), "/content/drive/MyDrive/best_metric_files/best_metric_model_classification3d_array.pth")
                  print("saved new best metric model")
              print(
                  "current epoch: {} current accuracy: {:.4f} best accuracy: {:.4f} at epoch {}".format(
                      epoch + 1, metric, best_metric, best_metric_epoch
                  )
              )
              writer.add_scalar("val_accuracy", metric, epoch + 1)
              
print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
writer.close()

# Evaluation Part

In [ ]:
import logging
import os
import sys

import numpy as np
import torch
from torch.utils.data import DataLoader

import monai
from monai.data import CSVSaver, ImageDataset
from monai.transforms import AddChannel, Compose, Resize, ScaleIntensity, EnsureType

In [ ]:
monai.config.print_config()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


images = []
labels = []

image_labels= pd.read_csv("/content/drive/MyDrive/test.csv")
#labels = np.array([], dtype=np.int64)
for index in range(40):
  labels.append(label_to_class[image_labels.iloc[index, 1]])
  images.append(os.path.join("/content/drive/MyDrive/preprocessed/test",
                                    image_labels.iloc[index,0]))  #image_labels.iloc[index,1],

labels = np.array(labels)
# Define transforms for image
val_transforms = Compose([ScaleIntensity(), AddChannel(),  EnsureType()]) 

# Define image dataset
val_ds = ImageDataset(image_files=images, labels=labels, transform=val_transforms, image_only=False)
# create a validation data loader
val_loader = DataLoader(val_ds, batch_size=5, num_workers=4, pin_memory=torch.cuda.is_available())

# Create DenseNet121 based model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=3).to(device) #Out channels önemli  DenseNet121

model.load_state_dict(torch.load("/content/drive/MyDrive/best_metric_files/best_metric_model_classification3d_array_4.pth"))
model.eval()
with torch.no_grad():
    num_correct = 0.0
    metric_count = 0
    saver = CSVSaver(output_dir="./output")
    for val_data in val_loader:
        val_images, val_labels = val_data[0].to(device), val_data[1].to(device)
        val_outputs = model(val_images).argmax(dim=1)
        value = torch.eq(val_outputs, val_labels)
        metric_count += len(value)
        num_correct += value.sum().item()
        saver.save_batch(val_outputs, val_data[2])
    metric = num_correct / metric_count
    print("evaluation metric:", metric)
    print("Number of Corrects:",num_correct)
    saver.finalize()